In [2]:
#!pip install transformers==4.45.0 accelerate==0.34.1 sentencepiece==0.2.0 torchvision requests torch Pillow

In [4]:
#!pip install flash-attn --no-build-isolation

In [6]:
#!pip install grouped_gemm==0.1.6

In [7]:
import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor

In [8]:
model_id_or_path = "rhymes-ai/Aria"

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_id_or_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

modeling_aria.py:   0%|          | 0.00/25.2k [00:00<?, ?B/s]

moe_lm.py:   0%|          | 0.00/24.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/rhymes-ai/Aria:
- moe_lm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


projector.py:   0%|          | 0.00/6.62k [00:00<?, ?B/s]

...

model-00011-of-00012.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

AriaMoELMForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [10]:
processor = AutoProcessor.from_pretrained(model_id_or_path, trust_remote_code=True)

preprocessor_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

processing_aria.py:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

vision_processor.py:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/rhymes-ai/Aria:
- vision_processor.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/rhymes-ai/Aria:
- processing_aria.py
- vision_processor.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [11]:
urls = [
 'https://samplebook.photos/img/4786a.jpg',
 'https://samplebook.photos/img/284v.jpg',
 'https://samplebook.photos/img/3193c.jpg',
 'https://samplebook.photos/img/948d.jpg',
 'https://samplebook.photos/img/3191i.jpg',
 'https://samplebook.photos/img/4788oo.jpg',
 'https://samplebook.photos/img/2086d.jpg',
 'https://samplebook.photos/img/4781r.jpg',
 'https://samplebook.photos/img/5521n.jpg',
 'https://samplebook.photos/img/2073cc.jpg',
 'https://samplebook.photos/img/2085b.jpg',
 'https://samplebook.photos/img/4782c.jpg',
 'https://samplebook.photos/img/4783v.jpg',
 'https://samplebook.photos/img/4912gg.jpg',
 'https://samplebook.photos/img/304h.jpg',
 'https://samplebook.photos/img/2078i.jpg',
 'https://samplebook.photos/img/2077q.jpg',
 'https://samplebook.photos/img/3192h.jpg',
 'https://samplebook.photos/img/904x.jpg',
 'https://samplebook.photos/img/4792b.jpg',
 'https://samplebook.photos/img/5301p.jpg',
 'https://samplebook.photos/img/5207j.jpg',
 'https://samplebook.photos/img/1007p.jpg',
 'https://samplebook.photos/img/17l.jpg',
 'https://samplebook.photos/img/286t.jpg',
 'https://samplebook.photos/img/287g.jpg',
 'https://samplebook.photos/img/3189j.jpg',
 'https://samplebook.photos/img/4911d.jpg',
 'https://samplebook.photos/img/4785a.jpg',
 'https://samplebook.photos/img/4793a.jpg'
]

In [12]:
prompt = 'There are two objects in this image. One is a color checker, and the other is a photographic sample book, open to a page containing a photograph. Give a detailed description of the photograph.'

In [13]:
def get_response(url):
    
    image = Image.open(requests.get(url, stream=True).raw)
    
    messages = [
        {
            "role": "user",
            "content": [
                {"text": None, "type": "image"},
                {"text": prompt, "type": "text"},
            ],
        }
    ]
    
    text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=text, images=image, return_tensors="pt")
    inputs["pixel_values"] = inputs["pixel_values"].to(model.dtype)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.inference_mode(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        output = model.generate(
            **inputs,
            max_new_tokens=1024,
            stop_strings=["<|im_end|>"],
            tokenizer=processor.tokenizer,
            do_sample=True,
            temperature=0.9,
        )
        output_ids = output[0][inputs["input_ids"].shape[1]:]
        result = processor.decode(output_ids, skip_special_tokens=True)

    return result

In [14]:
responses_aria = {}

for url in urls:
    response = get_response(url)
    print(url,"\n",response,"\n\n")
    responses_aria[url] = {'rhymes-ai/Aria':response}

/tmp/ipykernel_89/1589659609.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.inference_mode(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


https://samplebook.photos/img/4786a.jpg 
 The photograph in the sample book shows a black and white image of a rocky landscape or cliffside. The photograph captures a dramatic scene with sharp contrasts, emphasizing the textures of the rock formations. Some vegetation can be seen scattered across the rocky surface, adding to the image's natural beauty and complexity. The photograph is bordered by a small white margin, and there is text on the right side of the image indicating "GEVAERT" and “RIDAX 5”.<|im_end|> 


https://samplebook.photos/img/284v.jpg 
 The photograph in the sample book appears to depict an industrial scene, most likely a power plant. It features several tall, cylindrical structures, possibly cooling towers, with narrow black and white patterns extending from the top. In the background, there are intricate metal frameworks and crisscrossing cables or pipelines, indicative of a large-scale power generation or distribution facility. The composition is striking, with the

In [15]:
import pickle

In [16]:
with open("responses_aria.pkl", "wb") as f:
    pickle.dump(responses_aria, f)